In [3]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
np.seterr(divide='ignore')
lista_filtrada = []
#lista=md.filtradodemonedas ()
lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 
         'ZRXUSDT', 'OMGUSDT', 'DOGEUSDT', 'WAVESUSDT', 'MKRUSDT', 'DOTUSDT', 'YFIUSDT', 'TRBUSDT', 'RUNEUSDT', 'SOLUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'DYDXUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 
         'INJUSDT', 'LDOUSDT', 'APTUSDT', 'ARBUSDT', 'SUIUSDT', 'WLDUSDT', 'ARKMUSDT', 'SEIUSDT', 'CYBERUSDT', 'ARKUSDT', 'GASUSDT', 
         'TIAUSDT', 'CAKEUSDT', 'MEMEUSDT', 'ORDIUSDT', 'NTRNUSDT']
lista = ['LINKUSDT']

timeframe = '15m'
if len(lista) > 1:
    imprimo = False
    debug = False
else:
    imprimo = True
    debug = True
print(lista)
porcentajes_sumados = 0
win_rate_buenos = 0
win_rate_malos = 0
trades = 0
balance = md.balancetotal()
for symbol in lista:    
    try:
        #data = md.estrategia_smart(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe, balance = balance, largo = 1)
        data = md.estrategia_alex(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe, balance = balance, largo = 1)
        resultado = md.backtesting_smart(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
        porcentajes_sumados=porcentajes_sumados+resultado['Return [%]']
        trades = trades + resultado['# Trades']
        if not np.isnan(resultado['Win Rate [%]']):
            if resultado['Win Rate [%]'] >= 50:
                win_rate_buenos = win_rate_buenos+1
            else:
                win_rate_malos = win_rate_malos+1
        # para smart
        #if ((data.Close.iloc[-1] < (data.decisional_alcista_high.iloc[-2] + (data.atr.iloc[-1]*3)))
        #    or 
        #    (data.Close.iloc[-1] > (data.decisional_bajista_low.iloc[-2] - (data.atr.iloc[-1]*3)))
        #    ):
        #    print(f"posible entrada symbol: {symbol}")
        # para alex
        if (    data.cruce_bos.iloc[-1] != -2
            or  data.cruce_bos.iloc[-2] != -2  
            or  data.cruce_bos.iloc[-3] != -2
            or  data.cruce_bos.iloc[-4] != -2
            or  data.cruce_bos.iloc[-5] != -2
            ):
            print(f"posible entrada symbol: {symbol}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
print(f"\nTimeframe {timeframe} - porcentajes_sumados {md.truncate(porcentajes_sumados,2)} - trades {trades} - win_rate_buenos {win_rate_buenos} - win_rate_malos {win_rate_malos} - Ganancia por trade: {md.truncate((porcentajes_sumados/trades if trades !=0 else porcentajes_sumados),2)}%")


['LINKUSDT']


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


LINKUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Timeframe 15m - porcentajes_sumados 0.0 - trades 0 - win_rate_buenos 0 - win_rate_malos 0 - Ganancia por trade: 0.0%


In [12]:
data['Open Time'].dt.dayofweek.iloc[-1]

3